# Retrieving retweets from Twitter API
The process of getting all retweets of a tweet is not straight forward, as there seem to be a lot of retweets out of reach.

According to documentation for [GET statuses/retweeters/ids][uids]:
    
> Returns a collection of up to 100 user IDs belonging to users who have retweeted the Tweet specified by the id parameter.

Likewise, for [GET statuses/retweets/:id][tweets]:
> Returns a collection of the 100 most recent retweets of the Tweet specified by the id parameter.

However, in spite of this information, there seems to be more to it. The following tweet is an example of this.

[uids]: https://dev.twitter.com/rest/reference/get/statuses/retweeters/ids
[tweets]: https://dev.twitter.com/rest/reference/get/statuses/retweets/id

## Example tweet
Below is an example of a tweet embedded.


In [1]:
tweet_id = 890179951403683845

In [2]:
%%html
<blockquote class="twitter-tweet" data-lang="en"><p lang="en" dir="ltr">New footage has experts questioning the long held theory that in the shark hierarchy, size trumps all <a href="https://t.co/9jUXA0uiCy">https://t.co/9jUXA0uiCy</a></p>&mdash; National Geographic (@NatGeo) <a href="https://twitter.com/NatGeo/status/890179951403683845">July 26, 2017</a></blockquote>
<script async src="//platform.twitter.com/widgets.js" charset="utf-8"></script>

## Code

In [3]:
import sys
print(sys.version)

3.5.3 |Continuum Analytics, Inc.| (default, May 15 2017, 10:43:23) [MSC v.1900 64 bit (AMD64)]


In [4]:
from TwitterAPI import TwitterAPI, TwitterRequestError, TwitterConnectionError, TwitterRestPager
from pprint import pprint
from urllib.parse import urlencode, quote_plus
from collections import defaultdict

from datetime import datetime
import dateutil.parser

In [5]:
# Local module to keep secrets out of the notebook
from auth.secrets import TwitterSecrets

In [6]:
key, secret = TwitterSecrets.key, TwitterSecrets.secret
user_token, user_secret = TwitterSecrets.user_token, TwitterSecrets.user_token_secret

api = TwitterAPI(key, secret, user_token, user_secret, auth_type='oAuth1')

In [7]:
def get_tweet(tweet_id):
    return api.request('statuses/show/:{}'.format(tweet_id)).json()


def get_retweeters(tweet_id):
    """Get id of user who has reetweeted the given tweet"""
    return api.request('statuses/retweeters/ids', {
        'id': tweet_id, 
        'stringify_ids': True
    }).json()


def get_retweets(tweet_id):
    """Get id of reetweeted messages of the given tweet"""
    return api.request('statuses/retweets/:{}'.format(tweet_id)).json()

In [8]:
tweet = get_tweet(tweet_id)

In [9]:
retweet_count = tweet.get('retweet_count')
retweeters = get_retweeters(tweet_id)
retweets = get_retweets(tweet_id)

In [10]:
print('Actual retweet count: {:>18n}'.format(retweet_count))
print('Number of fetched retweeters: {:>10n}'.format(len(retweeters.get('ids'))))
print('Number of fetched retweets: {:>12n}'.format(len(retweets)))

Actual retweet count:                 41
Number of fetched retweeters:         37
Number of fetched retweets:           16


In [11]:
user_ids_from_retweets = set()

for tweet in retweets:
    user_id = tweet.get('user',{}).get('id', 'NO_ID')
    user_ids_from_retweets.add(user_id)

# Print
print(sorted(user_ids_from_retweets))

[22200055, 41803013, 81506989, 147642804, 573053124, 937856665, 944111292, 1869026826, 3296674325, 733733770680016901, 771744369560936450, 771830370736017408, 806828291957125120, 861477967859470336, 868828841904455680, 875326658475065344]


In [12]:
user_ids_from_retweeters = set(int(uid) for uid in retweeters.get('ids'))
print(sorted(user_ids_from_retweeters))

[22200055, 41803013, 81506989, 147642804, 519726574, 573053124, 707874535, 937856665, 944111292, 972936630, 1127391626, 1869026826, 2245822809, 2432168208, 2803245280, 3296674325, 4742504360, 733733770680016901, 749992566003761152, 771744369560936450, 771830370736017408, 806828291957125120, 861477967859470336, 862175763273043969, 862178030852210688, 862180504191332352, 862182361492008960, 862184657911255040, 864195290047426560, 868828841904455680, 869484430880100353, 869485177394823168, 869485815453372416, 875326658475065344, 876769092375642112, 876776659134947328, 876784964859301890]


In [13]:
# All user ids from retweets are in the bigger set of retweeters if this is an empty set:
len(user_ids_from_retweets - user_ids_from_retweeters)

0

In [14]:
# Number of retweeters which we don't have the tweet for:
len(user_ids_from_retweeters - user_ids_from_retweets)

21

## Raw data from Twitter
This is just kept at the bottom for the curious ones.

In [15]:
pprint(retweeters)

{'ids': ['733733770680016901',
         '771830370736017408',
         '1869026826',
         '573053124',
         '806828291957125120',
         '875326658475065344',
         '3296674325',
         '944111292',
         '81506989',
         '861477967859470336',
         '937856665',
         '22200055',
         '147642804',
         '41803013',
         '868828841904455680',
         '771744369560936450',
         '972936630',
         '749992566003761152',
         '707874535',
         '519726574',
         '864195290047426560',
         '2432168208',
         '4742504360',
         '2245822809',
         '876776659134947328',
         '869485177394823168',
         '869484430880100353',
         '862182361492008960',
         '862180504191332352',
         '876784964859301890',
         '862175763273043969',
         '876769092375642112',
         '862184657911255040',
         '869485815453372416',
         '862178030852210688',
         '1127391626',
         '2803245280'],
 

In [16]:
pprint(retweets)

[{'contributors': None,
  'coordinates': None,
  'created_at': 'Wed Jul 26 12:07:13 +0000 2017',
  'entities': {'hashtags': [],
               'symbols': [],
               'urls': [{'display_url': 'on.natgeo.com/2h2dnBB',
                         'expanded_url': 'http://on.natgeo.com/2h2dnBB',
                         'indices': [114, 137],
                         'url': 'https://t.co/9jUXA0uiCy'}],
               'user_mentions': [{'id': 17471979,
                                  'id_str': '17471979',
                                  'indices': [3, 10],
                                  'name': 'National Geographic',
                                  'screen_name': 'NatGeo'}]},
  'favorite_count': 0,
  'favorited': False,
  'geo': None,
  'id': 890181711849566208,
  'id_str': '890181711849566208',
  'in_reply_to_screen_name': None,
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'is_quote_sta

 {'contributors': None,
  'coordinates': None,
  'created_at': 'Wed Jul 26 12:07:08 +0000 2017',
  'entities': {'hashtags': [],
               'symbols': [],
               'urls': [{'display_url': 'on.natgeo.com/2h2dnBB',
                         'expanded_url': 'http://on.natgeo.com/2h2dnBB',
                         'indices': [114, 137],
                         'url': 'https://t.co/9jUXA0uiCy'}],
               'user_mentions': [{'id': 17471979,
                                  'id_str': '17471979',
                                  'indices': [3, 10],
                                  'name': 'National Geographic',
                                  'screen_name': 'NatGeo'}]},
  'favorite_count': 0,
  'favorited': False,
  'geo': None,
  'id': 890181689124691968,
  'id_str': '890181689124691968',
  'in_reply_to_screen_name': None,
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'is_quote_sta

                         'url': 'https://t.co/9jUXA0uiCy'}],
               'user_mentions': [{'id': 17471979,
                                  'id_str': '17471979',
                                  'indices': [3, 10],
                                  'name': 'National Geographic',
                                  'screen_name': 'NatGeo'}]},
  'favorite_count': 0,
  'favorited': False,
  'geo': None,
  'id': 890181368776318976,
  'id_str': '890181368776318976',
  'in_reply_to_screen_name': None,
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'is_quote_status': False,
  'lang': 'en',
  'place': None,
  'possibly_sensitive': False,
  'retweet_count': 41,
  'retweeted': False,
  'retweeted_status': {'contributors': None,
                       'coordinates': None,
                       'created_at': 'Wed Jul 26 12:00:13 +0000 2017',
                       'entities': {'hashtags': [],
          

           'protected': False,
           'screen_name': 'MargosunC',
           'statuses_count': 111750,
           'time_zone': None,
           'translator_type': 'none',
           'url': None,
           'utc_offset': None,
           'verified': False}},
 {'contributors': None,
  'coordinates': None,
  'created_at': 'Wed Jul 26 12:05:50 +0000 2017',
  'entities': {'hashtags': [],
               'symbols': [],
               'urls': [{'display_url': 'on.natgeo.com/2h2dnBB',
                         'expanded_url': 'http://on.natgeo.com/2h2dnBB',
                         'indices': [114, 137],
                         'url': 'https://t.co/9jUXA0uiCy'}],
               'user_mentions': [{'id': 17471979,
                                  'id_str': '17471979',
                                  'indices': [3, 10],
                                  'name': 'National Geographic',
                                  'screen_name': 'NatGeo'}]},
  'favorite_count': 0,
  'favorited': False,
 

           'id': 875326658475065344,
           'id_str': '875326658475065344',
           'is_translation_enabled': False,
           'is_translator': False,
           'lang': 'sv',
           'listed_count': 0,
           'location': 'Malmoe, Sweden',
           'name': 'Marko Kraljevic',
           'notifications': False,
           'profile_background_color': 'F5F8FA',
           'profile_background_image_url': None,
           'profile_background_image_url_https': None,
           'profile_background_tile': False,
           'profile_image_url': 'http://pbs.twimg.com/profile_images/875328254613344256/HgKtEVIS_normal.jpg',
           'profile_image_url_https': 'https://pbs.twimg.com/profile_images/875328254613344256/HgKtEVIS_normal.jpg',
           'profile_link_color': '1DA1F2',
           'profile_sidebar_border_color': 'C0DEED',
           'profile_sidebar_fill_color': 'DDEEF6',
           'profile_text_color': '333333',
           'profile_use_background_image': True,
        

...

           'description': 'วาไรตี้ อย่าซีเรียส ส่วนเอกลักษณ์น่ะหรอ '
                          'ความไม่มีเอกลักษณ์น่ะแหละ อิอิ 😁',
           'entities': {'description': {'urls': []}},
           'favourites_count': 2571,
           'follow_request_sent': False,
           'followers_count': 1042,
           'following': False,
           'friends_count': 899,
           'geo_enabled': True,
           'has_extended_profile': False,
           'id': 81506989,
           'id_str': '81506989',
           'is_translation_enabled': False,
           'is_translator': False,
           'lang': 'th',
           'listed_count': 0,
           'location': '',
           'name': 'หดหู่ไม่หยุด',
           'notifications': False,
           'profile_background_color': '000000',
           'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme19/bg.gif',
           'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme19/bg.gif',
           'profile_back

                       'in_reply_to_status_id': None,
                       'in_reply_to_status_id_str': None,
                       'in_reply_to_user_id': None,
                       'in_reply_to_user_id_str': None,
                       'is_quote_status': False,
                       'lang': 'en',
                       'place': None,
                       'possibly_sensitive': False,
                       'retweet_count': 41,
                       'retweeted': False,
                       'source': '<a href="http://www.spredfast.com" '
                                 'rel="nofollow">Spredfast app</a>',
                       'text': 'New footage has experts questioning the long '
                               'held theory that in the shark hierarchy, size '
                               'trumps all https://t.co/9jUXA0uiCy',
                       'truncated': False,
                       'user': {'contributors_enabled': False,
                                'created_at

           'entities': {'description': {'urls': []},
                        'url': {'urls': [{'display_url': 'jeaninethurston.com',
                                          'expanded_url': 'http://www.jeaninethurston.com',
                                          'indices': [0, 23],
                                          'url': 'https://t.co/6kMaC9z72W'}]}},
           'favourites_count': 2390,
           'follow_request_sent': False,
           'followers_count': 1346,
           'following': False,
           'friends_count': 498,
           'geo_enabled': True,
           'has_extended_profile': False,
           'id': 22200055,
           'id_str': '22200055',
           'is_translation_enabled': False,
           'is_translator': False,
           'lang': 'en',
           'listed_count': 107,
           'location': 'Denver, Colorado',
           'name': 'Jeanine Thurston',
           'notifications': False,
           'profile_background_color': '214542',
           'profile

  'retweeted_status': {'contributors': None,
                       'coordinates': None,
                       'created_at': 'Wed Jul 26 12:00:13 +0000 2017',
                       'entities': {'hashtags': [],
                                    'symbols': [],
                                    'urls': [{'display_url': 'on.natgeo.com/2h2dnBB',
                                              'expanded_url': 'http://on.natgeo.com/2h2dnBB',
                                              'indices': [102, 125],
                                              'url': 'https://t.co/9jUXA0uiCy'}],
                                    'user_mentions': []},
                       'favorite_count': 58,
                       'favorited': False,
                       'geo': None,
                       'id': 890179951403683845,
                       'id_str': '890179951403683845',
                       'in_reply_to_screen_name': None,
                       'in_reply_to_status_id': None,
          

           'has_extended_profile': True,
           'id': 868828841904455680,
           'id_str': '868828841904455680',
           'is_translation_enabled': False,
           'is_translator': False,
           'lang': 'en',
           'listed_count': 2,
           'location': 'Darwin, Northern Territory',
           'name': 'Alison porter',
           'notifications': False,
           'profile_background_color': 'F5F8FA',
           'profile_background_image_url': None,
           'profile_background_image_url_https': None,
           'profile_background_tile': False,
           'profile_banner_url': 'https://pbs.twimg.com/profile_banners/868828841904455680/1499177556',
           'profile_image_url': 'http://pbs.twimg.com/profile_images/868832953622712320/hZoOHON1_normal.jpg',
           'profile_image_url_https': 'https://pbs.twimg.com/profile_images/868832953622712320/hZoOHON1_normal.jpg',
           'profile_link_color': '1DA1F2',
           'profile_sidebar_border_color': 'C0DEE

In [17]:
pprint(tweet)

{'contributors': None,
 'coordinates': None,
 'created_at': 'Wed Jul 26 12:02:23 +0000 2017',
 'entities': {'hashtags': [],
              'symbols': [],
              'urls': [{'display_url': 'on.natgeo.com/2h2dnBB',
                        'expanded_url': 'http://on.natgeo.com/2h2dnBB',
                        'indices': [114, 137],
                        'url': 'https://t.co/9jUXA0uiCy'}],
              'user_mentions': [{'id': 17471979,
                                 'id_str': '17471979',
                                 'indices': [3, 10],
                                 'name': 'National Geographic',
                                 'screen_name': 'NatGeo'}]},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 890180493941104643,
 'id_str': '890180493941104643',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en